In [1]:
import pandas as pd
import sqlite3

PATH = './data/'
DATABASE = PATH + 'database.sqlite'

conn = sqlite3.connect(DATABASE)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


For the four biggest leagues of Europe (France, England, Italy, Spain) get a list of all the available matches.
For each match, record the season, the league, the home team, the away team, and the final score

In [28]:
matches_per_league = pd.read_sql("""   
                                      SELECT Match.id as match_id, 
                                        Match.season as season,
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        stage, 
                                        date,
                                        home_team_goal,
                                        away_team_goal,
                                        HT.team_long_name as home_team_name,
                                        AT.team_long_name as away_team_name
                                        
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN  Team as HT on Match.home_team_api_id = HT.team_api_id
                                LEFT JOIN  Team as AT on Match.away_team_api_id = AT.team_api_id
                                
                                where country_name in ('Spain', 'France', 'England', 'Italy')
                               
                                ;""", conn)
matches_per_league.head()

,match_id,season,country_name,league_name,stage,date,home_team_goal,away_team_goal,home_team_name,away_team_name
0,1729,2008/2009,England,England Premier League,1,2008-08-17 00:00:00,1,1,Manchester United,Newcastle United
1,1730,2008/2009,England,England Premier League,1,2008-08-16 00:00:00,1,0,Arsenal,West Bromwich Albion
2,1731,2008/2009,England,England Premier League,1,2008-08-16 00:00:00,0,1,Sunderland,Liverpool
3,1732,2008/2009,England,England Premier League,1,2008-08-16 00:00:00,2,1,West Ham United,Wigan Athletic
4,1733,2008/2009,England,England Premier League,1,2008-08-17 00:00:00,4,2,Aston Villa,Manchester City


## Basic exploration of main characteristics of the data

In [36]:
print('Number of Matches:', len(matches_per_league))
print('Number of Teams: ', matches_per_league['home_team_name'].nunique(), matches_per_league['away_team_name'].nunique())
print('Number of games per season and league: ', matches_per_league.groupby(['country_name', 'season']).size())

Number of Matches: 12137
Number of Teams:  134 134
Number of games per season and league:  country_name  season   
England       2008/2009    380
              2009/2010    380
              2010/2011    380
              2011/2012    380
              2012/2013    380
              2013/2014    380
              2014/2015    380
              2015/2016    380
France        2008/2009    380
              2009/2010    380
              2010/2011    380
              2011/2012    380
              2012/2013    380
              2013/2014    380
              2014/2015    380
              2015/2016    380
Italy         2008/2009    380
              2009/2010    380
              2010/2011    380
              2011/2012    358
              2012/2013    380
              2013/2014    380
              2014/2015    379
              2015/2016    380
Spain         2008/2009    380
              2009/2010    380
              2010/2011    380
              2011/2012    380
              201

In [37]:
matches_per_league.to_csv('./data/matches_per_leaguge_to_four_leagues.csv', index=False)